In [1]:
#省书中的卷积，卷积核的文字介绍
import torch
from torch import nn
#输入X，卷积核K，输出Y
#这里是二位互相关运算
def corr2d(X,K):
    h,w = K.shape
    Y = torch.zeros(X.shape[0]-h+1,X.shape[1]-w+1)#Y的shape,这里默认的stride=1，padding=0
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
                    Y[i,j] = (X[i:i+h,j:j+w]*K).sum()#卷积运算实质
    return Y
    

In [2]:
#构造一个X，K进行验证
X = torch.tensor([[0,1,2],
                  [3,4,5],
                  [6,7,8]])
K = torch.tensor([[0,1],
                  [2,3]])
Y = corr2d(X,K)
print(Y)

tensor([[19., 25.],
        [37., 43.]])


In [3]:
#二维卷积层就是：将输入和卷积核做互相关运算，再加上一个标量bias的老输出
#所以卷积层的模型参数包括：卷积核+标量bias
#下面实现一个自定义的二维卷积层
class Conv2D(nn.Module):
    def __init__(self,kernel_size):
        super(Conv2D,self).__init__()
        self.weight = nn.Parameter(torch.randn(kernel_size))
        self.bias = nn.Parameter(torch.randn(1))
    def forward(self,x):
        return corr2d(x,self.weight)+self.bias

In [4]:
#下面进行一个卷积层的简单应用：检测图像中物体的边缘（找到像素变化的位置）
#先构造一张图片，大小是6*8，中间四列时0，其余时1
X = torch.ones(6,8)
X[:,2:6] = 0
X

tensor([[1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.]])

In [5]:
#构造一个卷积核，kernel_size为1*2
K = torch.tensor([[-1,1]])
Y = corr2d(X,K)
Y

tensor([[ 0., -1.,  0.,  0.,  0.,  1.,  0.],
        [ 0., -1.,  0.,  0.,  0.,  1.,  0.],
        [ 0., -1.,  0.,  0.,  0.,  1.,  0.],
        [ 0., -1.,  0.,  0.,  0.,  1.,  0.],
        [ 0., -1.,  0.,  0.,  0.,  1.,  0.],
        [ 0., -1.,  0.,  0.,  0.,  1.,  0.]])

In [6]:
conv2d = Conv2D(kernel_size=(1,2))
#模型实例化
step = 20
lr = 0.01
for i in range(step):
    Y_hat = conv2d(X)
    l = ((Y_hat - Y)**2).sum()
    l.backward()
    #梯度下降
    conv2d.weight.data -= lr*conv2d.weight.grad
    conv2d.bias.data -= lr*conv2d.bias.grad
    #梯度清零
    conv2d.weight.grad.fill_(0)
    conv2d.bias.grad.fill_(0)
    if(i+1)%5==0:
        print('step %d,loss %.3f' %(i+1,l.item()))

step 5,loss 2.165
step 10,loss 0.404
step 15,loss 0.090
step 20,loss 0.023


In [7]:
print("weight:",conv2d.weight.data)
print("bias:",conv2d.bias.data)

weight: tensor([[-0.9689,  0.9574]])
bias: tensor([0.0065])


填充和步幅

In [9]:
#定义一个函数来计算卷积层，他对输入和输出做相应的升维和降维
def comp_conv2d(conv2d,X):
    #（1，1）代表批次大小和通道数
    X = X.view((1,1)+X.shape)
    Y = conv2d(X)
    return Y.view(Y.shape[2:])#排除添加的前两维即批次和通道数


In [11]:
conv2d = nn.Conv2d(in_channels = 1,out_channels = 1,kernel_size = 3,padding =1)
#两侧分别填充1行,或列，所以在两侧一共填充两行或列
X = torch.rand(8,8)
comp_conv2d(conv2d,X).shape

torch.Size([8, 8])

In [12]:
#后面都是基础的内容省去了，LeNet，AlexNet，VGG，GoogLeNet在part1部分有总结，拍project部分也有实战代码，可以直接去看